<a href="https://colab.research.google.com/github/sandhrabijoy/_/blob/master/Pytorch_practice_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np,pandas as pd ,pylab as pl
import h5py,torch
from tensorflow import image as timage
from torchvision import transforms,utils
from torch.utils.data import DataLoader as tdl
from torch.utils.data import Dataset as tds
import torch.nn as tnn


In [3]:
from IPython.core.magic import register_line_magic
dev=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
class TData(tds):
  def __init__(self,X,y):
    self.X=torch.tensor(X,dtype=torch.float32)
    self.y=torch.tensor(y,dtype=torch.int32)
  def __getitem__(self,index):
    train_img,train_lbl=self.X[index],self.y[index]
    return train_img,train_lbl
  def __len__(self):
    return self.y.shape[0]